In [0]:
import pandas as pd
import numpy as np
import pickle
import gc
import warnings
warnings.filterwarnings('ignore')

In [0]:
# If executing in Google Colab
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path_1 = "/content/drive/My Drive/Word2Vec_Modelling_Srijan/Data Produced/word2vec_result_final_500.pickle"
path_2 = "/content/drive/My Drive/Word2Vec_Modelling_Srijan/Data Produced/gs_result_final_500.pickle"

data = pd.read_pickle((path_1))
# data = data.drop(columns = ['doc_id'])
X_pred = (pd.read_pickle(path_2))

# gc.collect()

In [0]:
data.shape

(642290, 502)

In [0]:
X_pred.shape

(9092, 501)

In [0]:
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.model_selection import train_test_split                                      #to split the dataset for training and testing
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

target = data.importance         
data.drop(['importance', 'doc_id'], axis=1, inplace=True)     # taking the training data features
                                 
# del data

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size = 0.3, random_state = 32) # in this our main data is split into train and test

del data, target
gc.collect()

0

In [0]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(449603, 500) (449603,) (192687, 500) (192687,)


### Model 1

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
import tensorflow as tf

x_train = tf.keras.utils.normalize(X_train, axis=1)
x_test = tf.keras.utils.normalize(X_test, axis=1)
print("Normalization Complete!")

model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.Flatten())   # input layer
model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.1))
model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.1))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.1))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))   # hidden layer 1
model.add(Dropout(0.1))


model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))   # output layer

model.compile(optimizer='adam', loss= 'binary_crossentropy', metrics=['accuracy'])
print("Model Ready")

model.fit(x_train, np.array(y_train), epochs=5, validation_data = (x_test, np.array(y_test)))

Normalization Complete!
Model Ready
Epoch 1/5

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

14051/14051 [==============================] - 52s 4ms/step - loss: 0.4889 - accuracy: 0.7616 - val_loss: 0.4751 - val_accuracy: 0.7701
Epoch 2/5
14051/14051 [==============================] - 51s 4ms/step - loss: 0.4754 - accuracy: 0.7704 - val_loss: 0.4784 - val_accuracy: 0.7671
Epoch 3/5
14051/14051 [==============================] - 51s 4ms/step - loss: 0.4688 - accuracy: 0.7744 - val_loss: 0.4700 - val_accuracy: 0.7732
Epoch 4/5
14051/14051 [===========

In [0]:
model.predict(X_test)

In [0]:
cutoff = np.arange(0.005, 0.995, 0.005)
f1 = []
acc = []

pred = (model.predict(X_test))

for i in cutoff:
    f1.append(f1_score(pred >= i, y_test))
    acc.append(accuracy_score(pred >= i, y_test))

# print(f1)

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x = cutoff, y = f1, mode='lines', name='f1'))
fig.add_trace(go.Scatter(x = cutoff, y = acc, mode='lines', name='accuracy'))
fig.show()

max_cutoff = cutoff[f1 == max(f1)]

In [0]:
print(max_cutoff, max(f1))

In [0]:
GS_sent = pd.read_pickle("/content/drive/My Drive/Word2Vec_Modelling_Srijan/Data Produced/gs_sent.pickle")
GS_sent.columns = ["Sents", "doc_id"]

X_pred.drop(columns = ['doc_id'], inplace = True)

predictions = (model.predict(X_pred) >= max_cutoff)
# probability_of_predictions = model.predict_proba(X_pred)

# this command gives the sentences that are detected important on Gold Standard Data
result = pd.DataFrame()

result = (GS_sent.iloc[predictions == 1])
result['probabilities'] = probability_of_predictions[predictions == 1][:,1]
result['doc_id'] = result['doc_id'] - 4946

# length filter of output results
length = []
for i in np.array(result['Sents']):
  length.append(len(i)>=50)

result['Sents'] = (result.iloc[length])['Sents']
result.dropna(inplace = True)

result

In [0]:
result.to_csv("content/drive/My Drive/Word2Vec_Modelling_Srijan/result_word2vec_best.csv")